In [2]:
%%writefile example2.txt
first 
second line
the third line
then a fourth line

Overwriting example2.txt


In [1]:
from pyspark import SparkContext

In [2]:
sc=SparkContext()

In [3]:
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [4]:
text_rdd=sc.textFile('example2.txt')

In [5]:
words=text_rdd.map(lambda line:line.split())

In [6]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [7]:
text_rdd.collect()

['first ', 'second line', 'the third line', 'then a fourth line']

In [8]:
text_rdd.flatMap(lambda x:x.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

In [9]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [11]:
services=sc.textFile('services.txt')

In [12]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [14]:
services.map(lambda x:x.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [23]:
clean=services.map(lambda x:x[1:] if x[0]=='#' else x)

In [24]:
clean=clean.map(lambda x:x.split())

In [25]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [28]:
pairs=clean.map(lambda line:(line[3],line[-1]))

In [29]:
reKey=pairs.reduceByKey(lambda amt1,amt2:float(amt1)+float(amt2))

In [30]:
reKey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [31]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [34]:
step1=clean.map(lambda line:(line[3],line[-1]))
step2=step1.reduceByKey(lambda amt1,amt2:float(amt1)+float(amt2))
step3=step2.filter(lambda x:not x[0]=='State')
step4=step3.sortBy(lambda x:x[1],ascending=False)
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]